### Thư viện

In [ ]:
import matplotlib.pyplot as plt
import PIL as pil
import numpy as np

from PIL import Image

### In ảnh

In [ ]:
def print_img(imgs, row, col):
    plt.figure(figsize=(20,15))
    for i in range(len(imgs)):
        plt.subplot(row, col, i+1)
        plt.imshow(imgs[i])
    plt.show()

### Lưu ảnh

In [ ]:
def save_img(imgs, file_name, function):
    [Image.fromarray(imgs[i]).save(file_name.rsplit('.')[0] + '_' + function[i] + '.png') for i in range(1, len(imgs))]

### Thay đổi độ sáng cho ảnh

In [ ]:
def brighten(img_d, brightvalue):
    return np.clip(img_d.astype('int64') + brightvalue, 0, 255).astype('uint8')

### Thay đổi độ tương phản

In [ ]:
def contrast(img_d, contrastvalue):
    f = (259*(contrastvalue + 255))/(255*(259 - contrastvalue))
    return np.clip((f*(img_d.astype('float64') - 128) + 128), 0, 255).astype('uint8')

### Lật ảnh (ngang - dọc)

    - Lật ngang

In [ ]:
def hflip(img_d):
    return img_d[:,::-1]

    - Lật dọc

In [ ]:
def vflip(img_d):
    return img_d[::-1]

### Chuyển đổi ảnh RGB thành ảnh xám/sepia

    - RGB -> grayscale

In [ ]:
def to_grayscale(img_d):
    grayscale = np.array([0.3, 0.59, 0.11])
    res = np.sum(img_d * grayscale, axis = -1).astype('uint8').reshape(img_d.shape[0], img_d.shape[1], 1)
    return np.repeat(res, img_d.shape[-1], axis=-1)

    - RGB -> sepia

In [ ]:
def to_sepia(img_d):
    sepiascale = np.array([[0.393, 0.769, 0.189],
                          [0.349, 0.686, 0.168],
                          [0.272, 0.534, 0.131]])
    
    new_red = np.clip(np.sum(img_d * sepiascale[0], axis=-1), 0, 255).astype('uint8').reshape(img_d.shape[0], img_d.shape[1], 1)
    new_green = np.clip(np.sum(img_d * sepiascale[1], axis=-1), 0, 255).astype('uint8').reshape(img_d.shape[0], img_d.shape[1], 1)
    new_blue = np.clip(np.sum(img_d * sepiascale[2], axis=-1), 0, 255).astype('uint8').reshape(img_d.shape[0], img_d.shape[1], 1)
    
    return np.concatenate([new_red, new_green, new_blue], axis=-1)

### Làm mờ/sắc nét ảnh

    - Làm mờ ảnh

In [ ]:
def blur(img_d):
    kernel = 1/16 * np.array([[1, 2, 1],
                              [2, 4, 2],
                              [1, 2, 1]])

    blured_img = np.copy(img_d).astype('float64')
    
    for i in range(1, img_d.shape[0] - 1):
        for j in range(1, img_d.shape[1] - 1):
            for k in range(img_d.shape[2]):
                blured_img[i, j, k]= (img_d[i-1: i+2, j-1 : j+2, k].astype('float64') * kernel).sum()
 
    return np.clip(blured_img, 0, 255).astype('uint8')
    

    - Làm sắc nét

In [ ]:
def sharpen(img_d):
    kernel = np.array([[0, -1, 0],
                       [-1, 5, -1],
                       [0, -1, 0]])

    blured_img = np.copy(img_d).astype('float64')
    
    for i in range(1, img_d.shape[0] - 1):
        for j in range(1, img_d.shape[1] - 1):
            for k in range(3):
                blured_img[i, j, k]= (img_d[i-1: i+2, j-1 : j+2, k].astype('float64') * kernel).sum()
    
    return np.clip(blured_img, 0, 255).astype('uint8')

### Cắt ảnh theo kích thước (cắt ở trung tâm)

In [ ]:
def crop_center(img_d):
    width = int(img_d.shape[0]/4)
    height = int(img_d.shape[1]/4)
    
    res = img_d[width:-width, height:-height]
    
    return res

### Cắt ảnh theo khung hình tròn

In [ ]:
def crop_circle(img_d):
    a = int(img_d.shape[0]/2)
    b = int(img_d.shape[1]/2)
    
    r = int(img_d.shape[0]/2)
    frame = np.copy(img_d)
    
    for i in range (img_d.shape[0]):
        for j in range(img_d.shape[1]):
            if ((i - a)**2 + (j - b)**2 > r**2):
                frame[i][j] = 0             
    return frame
    

### Cắt ảnh theo khung là 2 hình ellip chéo nhau

In [ ]:
def crop_ellip(img_d):
    h = int(img_d.shape[0]/2)
    k = int(img_d.shape[1]/2)
    
    a = int(np.sqrt(img_d.shape[0]**2 + img_d.shape[1]**2)/2.35)
    b = int(np.sqrt(img_d.shape[0]**2 + img_d.shape[1]**2)/4)
    
    x, y = np.meshgrid(np.arange(img_d.shape[1]), np.arange(img_d.shape[0]))
    
    x1 = ((x - h)*(np.sqrt(2)/2) + (y - k)*(np.sqrt(2)/2))**2/(a**2)
    y1 = ((x - h)*(np.sqrt(2)/2) - (y - k)*(np.sqrt(2)/2))**2/(b**2)
            
    x2 = ((x - h)*(-np.sqrt(2)/2) + (y - k)*(np.sqrt(2)/2))**2/(a**2)
    y2 = ((x - h)*(np.sqrt(2)/2) - (y - k)*(-np.sqrt(2)/2))**2/(b**2)
            
    mask = np.logical_and(x1 + y1 > 1, x2 + y2 > 1)
    
    frame = np.copy(img_d)
    frame[mask] = 0
    
    return frame


In [ ]:
def main():
    file_name = input('Nhập tên tập tin ảnh')
    img = Image.open(file_name)
    img_d = np.array(img)
    
    print("""
          0. Thực hiện tất cả chức năng
          1. Thay đổi độ sáng cho ảnh
          2. Thay đổi độ tương phản
          3. Lật ảnh ngang - dọc
          4. Chuyển đổi ảnh RGB thành ảnh xám và sepia
          5. Làm mờ và sắc nét ảnh
          6. Cắt ảnh theo kích thước (cắt ở trung tâm)
          7. Cắt ảnh theo khung hình tròn
          8. Cắt ảnh theo khung 2 hình ellip chéo nhau
          """)
        
    function = input('Nhập chức năng (0 - 8)')
    match function:
        case '0':
            imgs = [img, brighten(img_d, 128), contrast(img_d, 128), hflip(img_d), vflip(img_d),
            to_grayscale(img_d), to_sepia(img_d), blur(img_d), sharpen(img_d),
            crop_center(img_d), crop_circle(img_d), crop_ellip(img_d)]
            
            function_name = ['original', 'bright', 'constrast', 'flip horizontally',
                             'flip vertically', 'grayscale', 'sepia', 'blur', 'sharp',
                             'center crop', 'circle frame', 'ellip frame']
            
            print_img(imgs, 3, 4)
            save_img(imgs, file_name, function_name)
        case '1':
            imgs = [img, brighten(img_d, 128)]
            function_name = ['original', 'bright']
            
            print_img(imgs, 1, 2)
            save_img(imgs, file_name, function_name)
        case '2':
            imgs = [img, contrast(img_d, 128)]
            function_name = ['original', 'constrast']
            
            print_img(imgs, 1, 2)
            save_img(imgs, file_name, function_name)
        case '3':
            imgs = [img, vflip(img_d), hflip(img_d)]
            function_name = ['original', 'flip vertically', 'flip horizontally']
            
            print_img(imgs, 1, 3)
            save_img(imgs, file_name, function_name)
        case '4':
            imgs = [img, to_grayscale(img_d), to_sepia(img_d)]
            function_name = ['original', 'grayscale', 'sepia']
            
            print_img(imgs, 1, 3)
            save_img(imgs, file_name, function_name)
        case '5':
            imgs = [img, blur(img_d), sharpen(img_d)]
            function_name = ['original', 'blur', 'sharp']
            
            print_img(imgs, 1, 3)
            save_img(imgs, file_name, function_name)
        case '6':
            imgs = [img, crop_center(img_d)]
            function_name = ['original', 'center crop']
            
            print_img(imgs, 1, 2)
            save_img(imgs, file_name, function_name)
        case '7':
            imgs = [img, crop_circle(img_d)]
            function_name = ['original', 'circle frame']
            
            print_img(imgs, 1, 2)
            save_img(imgs, file_name, function_name)
        case '8':
            imgs = [img, crop_ellip(img_d)]
            function_name = ['original', 'ellip frame']
            
            print_img(imgs, 1, 2)
            save_img(imgs, file_name, function_name)
        case _:
            print('Chức năng không phù hợp')

In [ ]:
main()